# クリーニング部

In [97]:
# -*- coding: utf-8 -*-

## クリーニング処理
def cleaning_reviews(reviews):
    dust_keywords = ['<br />', ' ', '　', '【', '】', '(', ')','（',  '）','「', '」', '＋', '+','★', '☆', '※','~', '／', '〜', '*', '^', 'm(_ _)m', '(^^)', '(^^)', 'm(._.)m', 'ヽ=ﾟДﾟ=ﾉ']
    for key in dust_keywords:
        reviews['NEWS'] = reviews['NEWS'].replace('[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{6}\n', '', regex=True)
        reviews['NEWS'] = reviews['NEWS'].replace('[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}\+[0-9]{4}\n', '', regex=True)
        reviews['NEWS'] = reviews['NEWS'].replace('http(s*)://.+\n', '。', regex=True)
        reviews['NEWS'] = reviews['NEWS'].replace('iPhone', '携帯電話')
        reviews['NEWS'] = reviews['NEWS'].replace('\n', '。', regex=True)
        reviews['NEWS'] = reviews['NEWS'].str.replace(key, '')
    return reviews

In [ ]:
# -*- coding:utf-8 -*-

##### 当メイン関数は都度実行しなくてよい(クリーニング処理をし直したい場合のみ実行)

#### 前提条件
#・livedoorニュースコーパスをダウンロードして、解凍する。
#・textディレクトリ配下のニュースを、当JupyterNoteBookと同じ配下に配置する
#(∵GitHubにそのままコミットして公開の位置づけにならないための配慮)

## メイン関数
import glob
import codecs
import sys
import pandas as pd

count = 0
news_index = 1

df = pd.DataFrame([], columns=['CATEGORY', 'NEWS'])
count = count + 1
dlist1 = glob.glob("./text/*")
for dpath1 in dlist1:
    dlist2 = glob.glob(dpath1 + "/*")
    for dpath2 in dlist2:
        if 'LICENSE.txt' not in dpath2:
            allLines = codecs.open(dpath2, 'r', 'utf-8').read()
            df = df.append(pd.DataFrame([[news_index, allLines]], columns=['CATEGORY', 'NEWS'], index=[count]))
            count = count + 1
    news_index = news_index + 1

## ファイルの読込み(全件)
#shops = pd.read_csv('shops.txt', delimiter='\t')
#reviews = pd.read_csv('reviews.txt', delimiter='\t')

## クリーニング
df2 = cleaning_reviews(df)
df2.to_csv('newsmerge.txt')

## 確認用
#print(df2.NEWS)
#print(df2.loc[:, ['NEWS']])

# 機械学習部

In [ ]:
## Sudachiの初期化
import json
from sudachipy import tokenizer
from sudachipy import dictionary
from sudachipy import config

def init_sudachi():
    with open(config.SETTINGFILE, "r", encoding="utf-8") as f:
        settings = json.load(f)
    sudachi = dictionary.Dictionary(settings).create()
    mode = tokenizer.Tokenizer.SplitMode.C #モードC(一番長い形)に設定
    return sudachi, mode

In [ ]:
## 単語の分割
import json
from sudachipy import tokenizer
from sudachipy import dictionary
from sudachipy import config

def words_tokenize(comment, sudachi, mode):
    # tokenizeで変換した結果はオブジェクトなので、surfaceメソッドで形態素解析した結果を返却
    return [m.surface() for m in sudachi.tokenize(mode, comment)]

In [ ]:
import pandas as pd

## ラベルの作成
def makeLabel(points):
    result = pd.DataFrame(points)
    for i, point in enumerate(points):
        if point == 1:
            result.at[i, 'CATEGORY'] = 1
        elif point == 2:
            result.at[i, 'CATEGORY'] = 2
        elif point == 3:
            result.at[i, 'CATEGORY'] = 3
        elif point == 4:
            result.at[i, 'CATEGORY'] = 4
        elif point == 5:
            result.at[i, 'CATEGORY'] = 5
        elif point == 6:
            result.at[i, 'CATEGORY'] = 6
        elif point == 7:
            result.at[i, 'CATEGORY'] = 7
        elif point == 8:
            result.at[i, 'CATEGORY'] = 8
        else:
            result.at[i, 'CATEGORY'] = 9
    return result

In [ ]:
## メイン関数
import sys
import codecs
import random
import numpy as np
import numpy.random
import pandas as pd
import copy
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

# ファイルの読込み(全件)
#reviews = pd.read_csv('newsmerge.txt', delimiter=',')

# データ読込み
reviews = df

## Sudachi初期化
sudachi, mode = init_sudachi()

## 単語分割
tokenized_text_list = [words_tokenize(texts, sudachi, mode) for texts in reviews['NEWS']]

## 形態素解析した結果をシーケンス変換するクラスを生成(See. https://keras.io/ja/preprocessing/text/#tokenizer)
max_words = 100000 #一旦全部
tokenizer = Tokenizer(num_words=max_words)

## シーケンス変換するための事前学習
tokenizer.fit_on_texts(tokenized_text_list)

## シーケンス変換
seq = tokenizer.texts_to_sequences(tokenized_text_list)

## 入力値の整形(コメント側)
maxlen = 100 #100を超える場合は切り捨て、1000を満たない場合はゼロパティング
data = pad_sequences(seq, maxlen=maxlen)

## 入力値の整形(点数側)
labels = makeLabel(reviews['CATEGORY'])

categorical_labels = to_categorical(labels)
labels = np.asarray(categorical_labels)

print("Shape of data tensor:{}", format(data.shape)) # N件, maxlen
print("Shape of label tensor:{}", format(labels.shape)) # N件, 点数のカテゴリ化数

# 行列をランダムにシャッフル
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#labels = labels[indices]

## 評価データと検算データに分割
training_samples = data.shape[0] // 100 * 70 # "//" で商だけ取得可能
validation_samples = data.shape[0] // 100 * 90 # "//" で商だけ取得可能
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:validation_samples]
y_val = labels[training_samples:validation_samples]
x_predict = data[validation_samples:]
y_predict = labels[validation_samples:]

print("Shape of label tensor:{}", format(x_train.shape)) # N件, 点数のカテゴリ化数
print("Shape of label tensor:{}", format(x_val.shape)) # N件, 点数のカテゴリ化数
print("Shape of label tensor:{}", format(x_predict.shape)) # N件, 点数のカテゴリ化数

## 学習モデルの作成
model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(LSTM(32))
model.add(Dense(labels.shape[1], activation='sigmoid')) #点数のカテゴリ化数
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

## 学習開始
history = model.fit(x_train, y_train, epochs=30, batch_size=8, validation_split=0.2, validation_data=(x_val, y_val))

## モデルの保存
# model.sava('pre_trained_model.h5')

## 予測
results = model.predict(x_predict)
prediction_texts = ['1', '2', '3', '4', '5', '6', '7', '8', '9']


idx = range(1, x_predict.shape[0] + 1)
col = ["predict", "real"]
b = pd.DataFrame(index = idx, columns = col)

for i, res in enumerate(results):
#    print('予測：{0}　答え：{1}'.format(prediction_texts[np.argmax(res)], prediction_texts[np.argmax(y_predict[i])]))
    b.iat[i, 0] = prediction_texts[np.argmax(res)]
    b.iat[i, 1] = prediction_texts[np.argmax(y_predict[i])]
                                   
b.to_csv('c:\\temp\\yone.txt')


## 結果のプロット
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Trainig acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Trainig loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()





# メモ部

In [41]:
import pandas as pd

idx = range(1, 11)
col = ["predict", "real"]
a = pd.DataFrame(index = idx, columns = col)

print(a)
a.iat[1, 0] = 1
print(a)

print('hoge')
if "\u3000" in "abc\u3000def":
    print("true")
else:
    print("false")
#a.to_csv('c:\\temp\\yone.txt')

   predict real
1      NaN  NaN
2      NaN  NaN
3      NaN  NaN
4      NaN  NaN
5      NaN  NaN
6      NaN  NaN
7      NaN  NaN
8      NaN  NaN
9      NaN  NaN
10     NaN  NaN
   predict real
1      NaN  NaN
2        1  NaN
3      NaN  NaN
4      NaN  NaN
5      NaN  NaN
6      NaN  NaN
7      NaN  NaN
8      NaN  NaN
9      NaN  NaN
10     NaN  NaN
hoge
true


# Special Thanks

## 参考リンク

### "LSTMを使ってテキストの多クラス分類する".CODING ECHO
https://goo.gl/MGJHTB

### "Jupyter Notebook の Tips をまとめてみた".Tech Blog
https://adtech.cyberagent.io/techblog/archives/2317

### "Jupyter 知っておくと少し便利なTIPS集".Qiita
https://qiita.com/simonritchie/items/d7dccb798f0b9c8b1ec5

### "自然言語処理における前処理の種類とその威力 ".Qiita 
livedoorニュースの加工方法に関して解説してくれている  
https://qiita.com/Hironsan/items/2466fe0f344115aff177

### "ダウンロード - livedoor ニュースコーパス".株式会社ロンウイット 
一定加工を行ってくれているlivedoorニュースデータを無料提供してくれている  
https://www.rondhuit.com/download.html#ldcc

## Python文法及びpandasなどのライブラリ使い方メモ

### pandas
"Pandas のデータフレームを確認する".Python でデータサイエンス   
データフレームの行だけ表示する実装など  
https://pythondatascience.plavox.info/pandas/%E3%83%87%E3%83%BC%E3%82%BF%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E3%82%92%E7%A2%BA%E8%AA%8D

"Pandas のデータフレームに行や列 (カラム) を追加する".Python でデータサイエンス  
https://pythondatascience.plavox.info/pandas/pandas%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E3%81%AB%E8%A1%8C%E3%82%84%E5%88%97%E3%82%92%E8%BF%BD%E5%8A%A0%E3%81%99%E3%82%8B

## KerasなどのAI関連のライブラリ使い方メモ
### あああ

## JupyterNoteBookの使い方忘れそうな部分のメモ

### Markdownモード
セルを選択した状態で m を入力すれば対応可能
その後、記載した後、Shift + Enterでラベルに変換  
編集したい場合は、1～6で戻るのが筋がよさげ  
rやyでも編集モードに入れるが、Markdownからコード編集モードに戻るので、数値で編集モードに入った方がよさげ